In [13]:
from sys import exit
import pandas as pd
import datetime
from datetime import date
import calendar
import os

dir_name = os.getcwd
data1 = pd.read_csv(r'input_registered_students.csv')


data2 = pd.read_csv(r'input_attendance.csv')

n1 = len(data1)
n2 = len(data2)
students = []
names = [""]
studentnames = {}
for i in range(n1):
    t = data1.loc[i, "Roll No"]
    s = data1.loc[i, "Name"]
    studentnames[t] = s
    names.append(s)
    students.append(t)

dates = [""]
for i in range(n2):
    t = data2.loc[i, "Timestamp"]
    t = t[:10]
    d = datetime.datetime.strptime(t, "%d-%m-%Y")
    s = d.strftime('%Y-%m-%d')
    s = datetime.datetime.strptime(s, '%Y-%m-%d')
    if(calendar.day_name[s.weekday()] == 'Monday' or calendar.day_name[s.weekday()] == 'Thursday'):
        dates.append(s)

day_delta = datetime.timedelta(days=1)
ini = dates[1]
fin = dates[len(dates)-1]+day_delta
dates = [""]

for i in range((fin - ini).days):
    s = ini + i*day_delta
    if(calendar.day_name[s.weekday()] == 'Monday' or calendar.day_name[s.weekday()] == 'Thursday'):
        s = s.strftime('%Y-%m-%d')
        d = datetime.datetime.strptime(s, "%Y-%m-%d")
        s = d.strftime('%d-%m-%Y')
        if s not in dates:
            dates.append(s)

final = pd.DataFrame()
roll = [""]
roll.extend(students)
final['Roll'] = roll
final['Name'] = names
for d in dates:
    if d != '':
        l = ['A'for i in range(n1+1)]
        l[0] = " "
        final[d] = l
l = [len(dates)-1 for i in range(n1+1)]
l[0] = " "
final['Actual Lecture Taken'] = l
l = [0 for i in range(n1+1)]
l[0] = " "
final['Total Real'] = l
l = [0 for i in range(n1+1)]
l[0] = " "
final['% Attendance'] = l
for stur in students:
    roll = [""for i in range(len(dates))]
    roll[0] = stur
    name = [""for i in range(len(dates))]
    name[0] = studentnames[stur]
    df = pd.DataFrame()
    df['Date'] = dates
    df['Roll'] = roll
    df['Name'] = name
    tac = [0 for i in range(len(dates))]
    tac[0] = ""
    real = [0 for i in range(len(dates))]
    real[0] = ""
    dup = [0 for i in range(len(dates))]
    dup[0] = ""
    invalid = [0 for i in range(len(dates))]
    invalid[0] = ""
    absent = [1 for i in range(len(dates))]
    absent[0] = ""
    j = students.index(stur)+1
    for i in range(n2):
        t = data2.loc[i, "Timestamp"]
        t = t[:10]
        r = data2.loc[i, "Attendance"]
        r = r[:8]
        if t in dates and r == stur:
            ind = dates.index(t)
            tac[ind] += 1
            time = data2.loc[i, "Timestamp"]
            time = time[11:]
            if(time[0] == '1' and time[1] == '4') or time[:5] == '15:00':
                if real[ind] == 0:
                    real[ind] = 1
                    absent[ind] = 0
                    final.loc[j, t] = 'P'
                    final.loc[j, 'Total Real'] += 1
                else:
                    dup[ind] += 1
            else:
                invalid[ind] += 1
    for i in range(1, n1+1):
        a = final.loc[i, 'Total Real']
        b = len(dates)-1
        if(a != 0):
            c = round(a/b*100, 2)
            final.loc[i, '% Attendance'] = c
    df['Total Attendance Count'] = tac
    df['Real'] = real
    df['duplicate'] = dup
    df['Invalid'] = invalid
    df['Absent'] = absent
    t = "output/"+stur+".xlsx"
    df.to_excel(t, index=False)
final.to_excel('output/attendance_report_consolidated.xlsx', index=False)
